In [ ]:
def crc5_usb(data: bytes) -> int:
    """
    Compute USB CRC5 for an 11-bit token payload (address + endpoint).
    Returns a 5-bit integer.
    Polynomial: x^5 + x^2 + 1 (0b00101)
    """
    poly = 0b00101
    crc = 0b11111  # shift register initialized with all 1s

    # Only 11 LSBs are used for token packets
    value = int.from_bytes(data, 'little')
    for i in range(11):
        bit = (value >> (10 - i)) & 1
        msb = (crc >> 4) & 1
        crc = ((crc << 1) & 0x1F) | 0  # shift left
        if bit ^ msb:
            crc ^= poly
    # invert bits
    crc ^= 0x1F
    return crc

def crc16_usb(data: bytes) -> int:
    """
    Compute USB CRC16 for data packets (up to 1023 bytes).
    Polynomial: x^16 + x^15 + x^2 + 1 (0x8005)
    """
    poly = 0x8005
    crc = 0xFFFF  # shift register initialized with all 1s

    for byte in data:
        for i in range(8):
            bit = ((byte >> (7 - i)) & 1) ^ ((crc >> 15) & 1)
            crc = ((crc << 1) & 0xFFFF)
            if bit:
                crc ^= poly
    # invert bits
    crc ^= 0xFFFF
    return crc

def crc16_usb(data: bytes) -> int:
    """
    Compute USB CRC16 for data packets (up to 1023 bytes).
    Polynomial: x^16 + x^15 + x^2 + 1 (0x8005).
    Bit order: LSB first (per USB spec).
    Init: 0xFFFF. Final XOR (invert): 0xFFFF.
    """
    poly = 0xA001  # reflected 0x8005 because we feed LSB-first
    crc = 0xFFFF

    for byte in data:
        crc ^= byte
        for _ in range(8):
            if crc & 1:
                crc = (crc >> 1) ^ poly
            else:
                crc >>= 1
    return crc ^ 0xFFFF  # invert at the end


def check_usb_packet(hex_packet: str) -> bool:
    """
    Given a full USB packet in hex, check if the CRC at the end is valid.
    For simplicity, assumes the last 2 bytes are CRC16 if packet is data,
    last byte is CRC5 if token.
    """
    packet = bytes.fromhex(hex_packet.replace(" ", ""))
    pid = packet[1]  # assuming 2nd byte is PID
    # Example: if PID is 0xE1 (DATA1) or 0xC3 (DATA0), check CRC16
    if pid & 0xF0 in (0xC0, 0xD0, 0xE0, 0xF0):  # DATA0/DATA1/etc
        payload = packet[2:-2]  # data payload
        crc_recv = int.from_bytes(packet[-2:], 'little')
        return crc16_usb(payload) == crc_recv
    else:  # token packet: check CRC5
        payload = packet[2:-1]
        crc_recv = packet[-1] & 0x1F
        return crc5_usb(payload) == crc_recv


In [28]:
drum_data = """
6983E000 00000522 69C9DC00 00001E00 00001E4B 0204087F 7F7F7F00 00000000 00540000 00000002 00020002 000200C4 85
4B0000087F7F7F7F00000000000000000000000002000200020002008B6B
4B0204 087F7F7F 7F000000 00000054 00000000 00020002 00020002 00C485
4b1000087f7f7f7f000000000000000000000000020002000200020075c4
4b0404087f7f7f7f0000000000a8000000000000020002000200020038a1
4b0404087f7f7f7f00000000002a00000000000002000200020002003ac0
4b0804087f7f7f7f0000000054000000000000000200020002000200affc
4b0804087f7f7f7f000000002a0000000000000002000200020002000e7c
4b0104087f7f7f7f000000000000002a000000000200020002000200f7bc
4b0104087f7f7f7f00000000000000540000000002000200020002005e02
4b0104087f7f7f7f000000000000007e0000000002000200020002003968
4b0104087f7f7f7f00000000000000a80000000002000200020002000f3f
4b0204087f7f7f7f000000000000a8000000000002000200020002003879
4b0204087f7f7f7f0000000000002a00000000000200020002000200bafb
"""
packets = [bytes.fromhex(data) for data in drum_data.strip().split("\n")]

In [29]:
packets[0].hex()

'6983e0000000052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200c485'

In [30]:
payload = packets[0][2:-2]  # data payload
expected = packets[0][-2:]
actual = crc16_usb(payload).to_bytes(2, byteorder='little')
(expected.hex(), actual.hex())

('c485', 'd442')

In [36]:
for i in range(0,30):
    print(f"{i:02d}", "  "*i, (packets[0][i:-2]).hex(), expected.hex(), crc16_usb( packets[0][i:-2]).to_bytes(2, byteorder='little').hex())


00  6983e0000000052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 968c
01    83e0000000052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 0c02
02      e0000000052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 d442
03        000000052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 67c9
04          0000052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 c2a3
05            00052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 2892
06              052269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 18c1
07                2269c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 8d08
08                  69c9dc0000001e0000001e4b0204087f7f7f7f0000000000005400000000000200020002000200 c485 8add
09                 